In [1]:
import numpy as np

In [2]:
import pandas as pd
pd.__version__

u'0.17.1'

In [3]:
url = "https://en.wikipedia.org/wiki/List_of_national_parks_of_the_United_States"
df = pd.read_html(url, header=0)[0]
df.head()

,Name,Photo,Location,Date established[2][4],Area[2],Recreation Visitors (2014)[5],Description
0,Acadia,NaN,Maine 44°21′N 68°13′W﻿ / ﻿44.35°N 68.21°W﻿ / 4...,"000000001919-02-26-0000February 26, 1919","7008191779190394916♠47,389.67 acres (191.8 km2)",2563129,Covering most of Mount Desert Island and other...
1,American Samoa,NaN,American Samoa 14°15′S 170°41′W﻿ / ﻿14.25°S 17...,"000000001988-10-31-0000October 31, 1988","7007364217078016000♠9,000.00 acres (36.4 km2)",13953,The southernmost national park is on three Sam...
2,Arches,NaN,Utah 38°41′N 109°34′W﻿ / ﻿38.68°N 109.57°W﻿ / ...,"000000001929-04-12-0000April 12, 1929","7008309661325648497♠76,518.98 acres (309.7 km2)",1284767,"This site features more than 2,000 natural san..."
3,Badlands,NaN,South Dakota 43°45′N 102°30′W﻿ / ﻿43.75°N 102....,"000000001978-11-10-0000November 10, 1978","7008982398434864749♠242,755.94 acres (982.4 km2)",868094,"The Badlands are a collection of buttes, pinna..."
4,Big Bend,NaN,Texas 29°15′N 103°15′W﻿ / ﻿29.25°N 103.25°W﻿ /...,"000000001944-06-12-0000June 12, 1944","7009324219248177910♠801,163.21 acres (3,242.2 ...",314102,Named for the prominent bend in the Rio Grande...


In [4]:
locations = df['Location'].str.extract(u'(\D+) (\d+°\d+′[NS]) (\d+°\d+′[WE]).*')
locations.columns = ['State', 'lat', 'lon']

locations['lat'] = locations['lat'].str.replace(u'°', '.')
locations['lon'] = locations['lon'].str.replace(u'°', '.')

locations.loc[locations['lat'].str.endswith('S'), 'lat'] = '-' + locations['lat']
locations.loc[locations['lon'].str.endswith('W'), 'lon'] = '-' + locations['lon']

locations['lat'] = locations['lat'].str.slice_replace(start=-2)
locations['lon'] = locations['lon'].str.slice_replace(start=-2)
locations[['lat', 'lon']] = locations[['lat', 'lon']].astype(float)

locations.head()

,State,lat,lon
0,Maine,44.21,-68.13
1,American Samoa,-14.15,-170.41
2,Utah,38.41,-109.34
3,South Dakota,43.45,-102.30
4,Texas,29.15,-103.15


In [5]:
df = pd.concat([df, locations], axis=1)

In [6]:
import cesiumpy

In [7]:
v = cesiumpy.Viewer()
for i, row in df.iterrows():
    l = row['Recreation Visitors (2014)[5]']
    cyl = cesiumpy.Cylinder(position=[row['lon'], row['lat'], l / 2.], length=l,
                            topRadius=10e4, bottomRadius=10e4, material='aqua')
    v.entities.add(cyl)
v

In [8]:
v = cesiumpy.Viewer()
for i, row in df.iterrows():
    l = row['Recreation Visitors (2014)[5]']
    p= cesiumpy.Point(position=[row['lon'], row['lat'], 0],
                      pixelSize=np.sqrt(l / 10000), color='blue')
    v.entities.add(p)
v

In [9]:
v = cesiumpy.Viewer()
pin = cesiumpy.Pin()
for i, row in df.iterrows():
    l = row['Recreation Visitors (2014)[5]']
    b = cesiumpy.Billboard(position=[row['lon'], row['lat'], 0], image = pin, scale=0.4)
    v.entities.add(b)
v